### PennyLane Codes

In [6]:
import pennylane as qml
from pennylane import numpy as np

In [10]:
dev = qml.device(name='default.qubit', wires=2, shots=10000)  # initializing a "device"

In [15]:
@qml.qnode(device=dev)       # The qml decorator
def entanglement_circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    return qml.sample(op=qml.PauliZ(wires=0)), qml.sample(op=qml.PauliZ(wires=1))  # If |0>, then Z gate just gives 1.|0> and if |1>, Z gate gives -1.|1>

In [16]:
q0, q1 = entanglement_circuit()

In [17]:
q0

tensor([ 1, -1,  1, ..., -1, -1,  1], dtype=int64, requires_grad=True)

In [18]:
q1

tensor([ 1, -1,  1, ..., -1, -1,  1], dtype=int64, requires_grad=True)

In [20]:
(q0 == q1).sum()    # Since our circuit is entanglement circuit, both the qubits will always be the same

tensor(10000, requires_grad=True)

In [21]:
# defining another quantum device
dev1 = qml.device(name='default.qubit', wires=1, shots=10000)

# Running a QNode on this device
@qml.qnode(device=dev1)
def parametrized_circuit(parameter):
    qml.RX(phi=parameter, wires=0)
    return qml.sample(op=qml.PauliZ(wires=0))

In [22]:
q_rx = parametrized_circuit(parameter=np.pi/2)

In [23]:
q_rx

tensor([ 1, -1, -1, ..., -1, -1, -1], dtype=int64, requires_grad=True)

In [24]:
q_rx[1:100]

tensor([-1, -1,  1,  1, -1,  1, -1,  1,  1,  1, -1,  1, -1, -1, -1, -1,
        -1, -1,  1, -1,  1,  1,  1,  1, -1, -1, -1,  1, -1,  1,  1,  1,
         1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1, -1, -1,  1,  1,  1,
         1,  1,  1, -1, -1,  1,  1,  1,  1,  1, -1, -1, -1,  1,  1,  1,
         1, -1, -1, -1, -1,  1,  1,  1, -1,  1, -1, -1, -1,  1,  1, -1,
         1,  1,  1,  1,  1,  1,  1, -1, -1, -1,  1, -1,  1, -1, -1,  1,
         1,  1, -1], dtype=int64, requires_grad=True)

In [30]:
q_rx_new = parametrized_circuit(parameter=np.pi/4, shots=5) # default shots of 10000 as we defined above is overwritten for this cell

q_rx_new     # Each time this cell is run, it gives a different result...

tensor([ 1,  1,  1,  1, -1], dtype=int64, requires_grad=True)

In [27]:
# sum = 0
# for I in q_rx:
#     sum += I
# print(sum)

# ---------------rather than the for loop we could have just used the sum() function -------------------#
# print(q_rx.sum())

In [31]:
def ion_trap_cnot(wires):
    return [
        qml.RY(np.pi/2, wires=wires[0]),
        qml.IsingXX(np.pi/2, wires=wires),
        qml.RX(-np.pi/2, wires=wires[0]),
        qml.RY(-np.pi/2, wires=wires[0]),
        qml.RY(-np.pi/2, wires=wires[1])
    ]

In [32]:
ionTrap_CNOT = ion_trap_cnot(wires=[0,1])

ionTrap_CNOT

[RY(1.5707963267948966, wires=[0]),
 IsingXX(1.5707963267948966, wires=[0, 1]),
 RX(-1.5707963267948966, wires=[0]),
 RY(-1.5707963267948966, wires=[0]),
 RY(-1.5707963267948966, wires=[1])]

In [58]:
# As the CNOT gate normally has no decomposition, we can use default.qubit
# here for expository purposes.
dev2 = qml.device(
    'default.qubit', wires=2, custom_decomps={"CNOT" : ion_trap_cnot}, shots=100
)

@qml.qnode(dev2, expansion_strategy="device")
def run_cnot():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliX(wires=1))

In [59]:
my_cnot = run_cnot()

my_cnot

tensor(0.22, requires_grad=True)

In [67]:
print(qml.draw(qnode=run_cnot)())

0: ──H──RY(1.57)─╭IsingXX(1.57)──RX(-1.57)──RY(-1.57)─┤     
1: ──────────────╰IsingXX(1.57)──RY(-1.57)────────────┤  <X>


In [7]:
op = qml.SWAP(wires=[0,1])
op.decomposition()

[CNOT(wires=[0, 1]), CNOT(wires=[1, 0]), CNOT(wires=[0, 1])]

In [8]:
qml.is_hermitian(op=op)   # To see if our SWAP gate is Hermitian

True

In [9]:
qml.is_unitary(op=op)    # To see if our SWAP gate is unitary

True